# Hands On Machine Learning

## Tensor Flow

In [ ]:
!pip3 install tensorflow

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install --upgrade numpy

In [1]:
import numpy as np
np.__version__

'1.15.0'

In [3]:
import tensorflow as tf

In [5]:
x=tf.Variable(3, name='x')
y=tf.Variable(4,name='y')
f = x*x*y + y +2

In [7]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [9]:
sess.close()

In [10]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [11]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [16]:
sess= tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


/Users/sanghyub/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [17]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()


True

In [19]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [20]:
x2.graph is tf.get_default_graph()

False

In [21]:
tf.reset_default_graph()

In [22]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x*3

In [23]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [26]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)


10
15


In [28]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape

In [30]:
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')

XT = tf.transpose(X)

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [36]:
n_epochs = 1000
lr = 0.01

X= tf.constant(scaled_housing_data_plus_bias, dtype= tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0),name='theta')
y_pred = tf.matmul(X, theta, name = 'predictions')

In [39]:
error = y_pred -y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m*tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - lr * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE= 15.231111
Epoch 100 MSE= 0.83836275
Epoch 200 MSE= 0.6225865
Epoch 300 MSE= 0.5984199
Epoch 400 MSE= 0.5825411
Epoch 500 MSE= 0.5703497
Epoch 600 MSE= 0.5608858
Epoch 700 MSE= 0.55349857
Epoch 800 MSE= 0.5477026
Epoch 900 MSE= 0.5431325


In [40]:
def my_fun(a,b):
    z = 0
    for i in range(100):
        z = a*np.cos(z+i) + z*np.sin(b-i)
    return z

In [41]:
gradients = tf.gradients(mse, [theta])[0]

In [43]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
training_op = optimizer.minimize(mse)

In [45]:
optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)

In [47]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A+5
with tf.Session() as sess:
    B_val1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
    
    print(B_val1)

[[6. 7. 8.]]


In [48]:
print(B_val2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [49]:
X = tf.placeholder(tf.float32, shape= (None, n+1), name='X')
y= tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [50]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [51]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [52]:
best_theta

array([[ 2.0685525 ],
       [ 0.82963324],
       [ 0.11875443],
       [-0.26555082],
       [ 0.3057157 ],
       [-0.00450222],
       [-0.03932687],
       [-0.8998529 ],
       [-0.8705097 ]], dtype=float32)

In [54]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [55]:
reset_graph()

n_epochs = 1000                                                                       # 책에는 없습니다.
learning_rate = 0.01                                                                  # 책에는 없습니다.

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # 책에는 없습니다.
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # 책에는 없습니다.
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # 책에는 없습니다.
error = y_pred - y                                                                    # 책에는 없습니다.
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # 책에는 없습니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # 책에는 없습니다.
training_op = optimizer.minimize(mse)                                                 # 책에는 없습니다.

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())                                # 책에는 없습니다.
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

에포크 0 MSE = 9.161543
에포크 100 MSE = 0.7145006
에포크 200 MSE = 0.566705
에포크 300 MSE = 0.5555719
에포크 400 MSE = 0.5488112
에포크 500 MSE = 0.5436362
에포크 600 MSE = 0.5396294
에포크 700 MSE = 0.5365092
에포크 800 MSE = 0.5340678
에포크 900 MSE = 0.5321474


In [56]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

In [57]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # 책에는 없습니다.

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [58]:
saver = tf.train.Saver({"weights": theta})

In [59]:
reset_graph()
# 빈 그래프로 시작합니다

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # 그래프 구조를 로드합니다.
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # 책에는 없습니다.

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # 그래프 상태를 로드합니다.
    best_theta_restored = theta.eval() # 책에는 없습니다.

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [60]:
np.allclose(best_theta, best_theta_restored)

True